<a href="https://colab.research.google.com/github/AdamJuma133/Crop-Plant-Disease-Identifier-Project/blob/main/plant_disease_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
jawadali1045_20k_multi_class_crop_disease_images_path = kagglehub.dataset_download('jawadali1045/20k-multi-class-crop-disease-images')

print('Data source import complete.')

100%|██████████| 2.34G/2.34G [00:32<00:00, 78.0MB/s]

Extracting files...


Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importing required libraries

In [3]:
!pip3 install tensorflow numpy matplotlib keras opencv-python

In [4]:
import matplotlib.pyplot as plt
from PIL import Image
import random
import os

import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import random
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img
from keras.utils import plot_model
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import seaborn as sns
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

### DATA VISUALISATION

In [5]:
import os
import matplotlib.pyplot as plt
from PIL import Image

# Function to plot 5 images from each folder
def plot_images_from_directory(base_directory, title_prefix, images_per_folder=5):
    folders = os.listdir(base_directory)

    # Iterate over each folder in the base directory
    for folder in folders:
        folder_path = os.path.join(base_directory, folder)

        if os.path.isdir(folder_path):  # Ensure it's a directory
            images = os.listdir(folder_path)[:images_per_folder]  # Get first 'images_per_folder' images

            # Plot each image
            plt.figure(figsize=(15, 5))  # Adjust the figure size
            for i, img_file in enumerate(images):
                img_path = os.path.join(folder_path, img_file)

                try:
                    img = Image.open(img_path)  # Load image
                    plt.subplot(1, images_per_folder, i+1)  # Create subplots
                    plt.imshow(img)
                    plt.axis('off')  # Hide axis
                    plt.title(f"{title_prefix}: {folder}")  # Use folder name as label

                except Exception as e:
                    print(f"Error loading image {img_file}: {e}")

            # Display the images for this folder
            plt.show()

# Define the base directory path
base_directory = '/kaggle/input/20k-multi-class-crop-disease-images/Train/'

# Plot 5 images from each folder with the folder name as the label
plot_images_from_directory(base_directory, 'Disease')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/20k-multi-class-crop-disease-images/Train/'

### DATA ANALYSIS

In [6]:
import os
import matplotlib.pyplot as plt

# Function to count images in each folder
def count_images_in_folders(base_directory):
    image_counts = {}

    # Iterate over each folder in the base directory
    folders = os.listdir(base_directory)

    for folder in folders:
        folder_path = os.path.join(base_directory, folder)

        if os.path.isdir(folder_path):  # Ensure it's a directory
            num_images = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
            image_counts[folder] = num_images

    return image_counts

# Function to plot the image counts
def plot_image_counts(image_counts):
    # Sorting the dictionary by values (image count)
    sorted_image_counts = dict(sorted(image_counts.items(), key=lambda item: item[1], reverse=True))

    # Extract folder names and image counts
    folders = list(sorted_image_counts.keys())
    counts = list(sorted_image_counts.values())

    # Plotting the data
    plt.figure(figsize=(15, 8))  # Set figure size
    plt.barh(folders, counts, color='skyblue')  # Horizontal bar chart
    plt.xlabel('Number of Images')
    plt.ylabel('Folder Name')
    plt.title('Number of Images in Each Folder')
    plt.tight_layout()  # Adjust layout
    plt.show()

# Define the base directory path
base_directory = '/kaggle/input/20k-multi-class-crop-disease-images/Train/'

# Count images in each folder
image_counts = count_images_in_folders(base_directory)

# Plot the image counts
plot_image_counts(image_counts)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/20k-multi-class-crop-disease-images/Train/'

## Building the Model

In [ ]:
base_dir = '/kaggle/input/20k-multi-class-crop-disease-images/Train/'
#Standardization of data 1./255
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [ ]:

# Creating training data generator
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),  # Resizing images to 150x150
    batch_size=20,           # Loading 20 images at a time
    class_mode='categorical', # Since it's multi-class classification, use 'categorical'
    subset='training'         # Set as training data
)

# Creating validation data generator
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),  # Resizing images to 150x150
    batch_size=20,
    class_mode='categorical', # Multi-class classification
    subset='validation'       # Set as validation data
)


In [ ]:
# Class labels for the training data
train_class_labels = train_generator.class_indices
print("Training class labels:", train_class_labels)

# Class labels for the validation data
validation_class_labels = validation_generator.class_indices
print("Validation class labels:", validation_class_labels)

In [ ]:
import pandas as pd

# Get class labels for training data
train_class_labels = train_generator.class_indices

# Get class labels for validation data
validation_class_labels = validation_generator.class_indices

# Convert class labels to a DataFrame for table-like display
train_labels_df = pd.DataFrame(list(train_class_labels.items()), columns=['Class Name', 'Index'])
validation_labels_df = pd.DataFrame(list(validation_class_labels.items()), columns=['Class Name', 'Index'])

# Display tables
print("Training Class Labels:")
print(train_labels_df)

print("\nValidation Class Labels:")
print(validation_labels_df)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Activation

# Define the number of classes
n_classes = 42  # Update this to match your number of classes

model = Sequential()

# Input layer and first convolutional block
model.add(Conv2D(64, (3, 3), padding='same', input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# Second convolutional block
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# Third convolutional block
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# Fourth convolutional block
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# Global Average Pooling
model.add(GlobalAveragePooling2D())

# Fully connected layers
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Output layer
model.add(Dense(n_classes))
model.add(Activation('softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


In [ ]:
# Ensure train_steps and validation_steps_per_epoch are integers
train_steps = int(np.ceil(train_generator.samples / batch_size))
validation_steps_per_epoch = int(np.ceil(validation_generator.samples / batch_size))

print("Train steps:", train_steps)
print("Validation steps per epoch:", validation_steps_per_epoch)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps,  # Use calculated train_steps
    epochs=10,  # You can adjust the number of epochs as needed
    validation_data=validation_generator,
    validation_steps=validation_steps_per_epoch  # Use calculated validation_steps
)


In [ ]:
import tensorflow as tf

# Convert ImageDataGenerator to tf.data.Dataset
def generator_to_dataset(generator):
    dataset = tf.data.Dataset.from_generator(
        lambda: generator,
        output_types=(tf.float32, tf.float32),
        output_shapes=([None, 150, 150, 3], [None, len(generator.class_indices)])
    )
    return dataset

# Convert train and validation generators
train_dataset = generator_to_dataset(train_generator)
validation_dataset = generator_to_dataset(validation_generator)

# Repeat datasets to ensure enough data for steps
train_dataset = train_dataset.repeat()
validation_dataset = validation_dataset.repeat()

# Batch the datasets
train_dataset = train_dataset.batch(batch_size)
validation_dataset = validation_dataset.batch(batch_size)

# Calculate steps per epoch
train_steps = int(np.ceil(train_generator.samples / batch_size))
validation_steps_per_epoch = int(np.ceil(validation_generator.samples / batch_size))

print("Train steps:", train_steps)
print("Validation steps per epoch:", validation_steps_per_epoch)

# Train the model
history = model.fit(
    train_dataset,
    steps_per_epoch=train_steps,  # Use calculated train_steps
    epochs=10,  # You can adjust the number of epochs as needed
    validation_data=validation_dataset,
    validation_steps=validation_steps_per_epoch  # Use calculated validation_steps
)


In [ ]:
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

# Path to the dataset directory
dataset_dir = '/kaggle/input/20k-multi-class-crop-disease-images/Train/'
target_count = 200  # Target number of images per class

# Create an instance of ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Iterate over each class folder
for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)

    if os.path.isdir(class_dir):
        # List all images in the class folder
        image_files = [f for f in os.listdir(class_dir) if f.endswith('.jpg') or f.endswith('.png')]

        # Check if the number of images is less than the target count
        num_images = len(image_files)
        if num_images < target_count:
            num_images_needed = target_count - num_images

            # Create an augmented directory inside the class folder
            augmented_class_dir = os.path.join(class_dir, 'augmented')
            os.makedirs(augmented_class_dir, exist_ok=True)

            # Augment images to reach the target count
            for image_file in image_files:
                img_path = os.path.join(class_dir, image_file)
                img = load_img(img_path)
                x = img_to_array(img)
                x = np.expand_dims(x, axis=0)

                # Generate augmented images
                i = 0
                for batch in datagen.flow(x, batch_size=1, save_to_dir=augmented_class_dir, save_prefix='aug', save_format='jpeg'):
                    i += 1
                    if i >= num_images_needed:
                        break
            print(f"Augmented {class_name} to {target_count} images.")
